## Realizamos el preprocesamiento de los datos de mascotas
### Importamos las librerias correspondientes

In [14]:
import pandas as pd
import numpy as np

### Importamos los dataset correspondientes

In [15]:
dfe_2023=pd.read_csv("../input/db/sales/mascotas-2023.csv", parse_dates=["Date"])
dfe_2024=pd.read_csv("../input/db/sales/mascotas-2024.csv",parse_dates=["Date"])
dfe_2025=pd.read_csv("../input/db/sales/mascotas-2025.csv",parse_dates=["Date"])
locations=pd.read_csv("../output/locations_cleaned.csv")
products=pd.read_csv("../output/Productos.csv")

### Exploramos la estructura y nos enfocamos solo en las ventas

In [16]:
dfe_2023.head()

,Date,DocumentNumber,Type,LocationId,ProductId,Quantity,TotalCost,TotalPrice,TotalDiscount,LocationStockId
0,2023-06-05,3203000000053001,Sale,1663,A170056025061010,1,91.55068,148.10068,0.0,1669
1,2023-06-05,3203000000053001,Sale,1663,D133095022151010,1,9.42500,20.17704,0.0,1669
2,2023-06-05,3203000000053001,Sale,1663,N139095048105012,1,24.20340,39.72072,0.0,1669
3,2023-06-05,3203000000053001,Sale,1663,D223029041072010,1,8.36940,20.17704,0.0,1669
4,2023-06-05,3203000000053001,Sale,1663,A172058026110018,1,134.46836,222.14348,0.0,1669


In [17]:
dfe_2023[dfe_2023["Type"]!="Sale"]

,Date,DocumentNumber,Type,LocationId,ProductId,Quantity,TotalCost,TotalPrice,TotalDiscount,LocationStockId
58424,2023-01-07,4101000000034108,Return,1655,P27103710804201S,-1,-0.60320,-1.32704,0.000000,1655
58425,2023-01-07,4101000000035108,Return,1655,P27103710804201S,-1,-0.60320,-1.32704,0.000000,1655
58426,2023-01-12,4101000000037108,Return,1655,P27303407509001S,-1,-5.51928,-17.49280,0.000000,1655
58427,2023-01-14,4101000000038108,Return,1655,P27203609809101S,-1,-30.62748,-47.10992,0.000000,1655
58428,2023-01-15,4101000000040108,Return,1655,P27003806003001S,-1,-6.03200,-12.10924,-1.346418,1655
...,...,...,...,...,...,...,...,...,...,...
356592,2023-11-25,4102000000025136,Return,1686,P27103007606101S,-1,-3.34776,0.00000,-12.104399,1686
356593,2023-11-25,4102000000025136,Return,1686,P27603207602301S,-1,-3.42316,-13.45136,0.000000,1686
356594,2023-11-27,4102000000026136,Return,1686,P27503707209401S,-1,-3.93588,-13.45136,0.000000,1686
356595,2023-12-24,4102000000036136,Return,1686,P27803707212401S,-1,-6.09232,-13.45136,0.000000,1686


In [18]:
#Unimos los dataferames de ventas de los tres años
mascotas= pd.concat([dfe_2023, dfe_2024, dfe_2025], ignore_index=True)
mascotas=mascotas[mascotas["Type"]=="Sale"]

### Creamos nuevas columnas y unimos los datasets

In [19]:
mascotas["venta"]=mascotas["TotalPrice"]-mascotas["TotalDiscount"]
mascotas["ganancia"]=mascotas["TotalPrice"]-mascotas["venta"]

In [20]:
mascotas['Category'] = mascotas['ProductId'].map(products.set_index('idProd')['PCategory'])
mascotas['SubCategory'] = mascotas['ProductId'].map(products.set_index('idProd')['subCat'])
mascotas['General'] = mascotas['ProductId'].map(products.set_index('idProd')['General'])
mascotas['TipoTienda'] = mascotas['LocationStockId'].map(locations.set_index('id')['type'])
mascotas['DimensionesTienda'] = mascotas['LocationStockId'].map(locations.set_index('id')['tamanio'])
mascotas['Ciudad'] = mascotas['LocationStockId'].map(locations.set_index('id')['city'])
mascotas=mascotas[(mascotas["General"]=="MASCOTAS") ]

### Agrupamos por las variables de interes

In [21]:
#Estructurar el dataset final

df_agg = mascotas.groupby(['Date', 'LocationId', 'LocationStockId','Category', 'TipoTienda',
       'DimensionesTienda', 'Ciudad'], as_index=False).agg({
    'venta': 'sum',
    'Quantity': 'sum'
})
df_agg.head()

,Date,LocationId,LocationStockId,Category,TipoTienda,DimensionesTienda,Ciudad,venta,Quantity
0,2023-01-02,1655,1655,ALIMENTAR - ACCESORIOS,TIENDA,MEDIANO,SAMBORONDON,70.334191,6
1,2023-01-02,1655,1655,ALIMENTAR - COMIDA,TIENDA,MEDIANO,SAMBORONDON,458.181582,22
2,2023-01-02,1655,1655,ALIMENTAR - SNACKS,TIENDA,MEDIANO,SAMBORONDON,27.512389,190
3,2023-01-02,1655,1655,ANTIPARASITOS,TIENDA,MEDIANO,SAMBORONDON,54.921360,2
4,2023-01-02,1655,1655,ASEAR,TIENDA,MEDIANO,SAMBORONDON,320.845865,20


### Realizamos el feature engineering de las variables estacionales

In [22]:
#feature Engineering luego de la union de los dataframes
festivos = pd.to_datetime(['2023-01-02', '2023-02-20', '2023-02-21', '2023-04-07','2023-05-01','2023-05-26','2023-08-11','2023-10-09','2023-11-02','2023-11-03','2023-12-25'
                           , '2024-01-01', '2024-02-12', '2024-02-23', '2024-03-29','2024-05-03','2024-05-24','2024-08-09','2024-10-11','2024-11-01','2024-11-04','2024-12-25',
                           '2025-01-01', '2025-03-03', '2025-03-04', '2025-04-18','2025-05-02','2025-05-23','2025-08-11','2025-10-10','2025-11-03','2025-11-04','2025-12-25'])
df_agg['es_festivo'] = df_agg['Date'].isin(festivos).astype(int)
df_agg['dia_semana'] = df_agg['Date'].dt.dayofweek
df_agg['mes'] = df_agg['Date'].dt.month
# Codificación cíclica semanal
df_agg['sin_semana'] = np.sin(2 * np.pi * df_agg['dia_semana'] / 7)
df_agg['cos_semana'] = np.cos(2 * np.pi * df_agg['dia_semana'] / 7)

# Codificación cíclica mensual
df_agg['sin_mes'] = np.sin(2 * np.pi * df_agg['mes'] / 12)
df_agg['cos_mes'] = np.cos(2 * np.pi * df_agg['mes'] / 12)

# Rezagos
df_agg['ventas_lag1'] = df_agg['venta'].shift(1)
df_agg['ventas_lag7'] = df_agg['venta'].shift(7)
df_agg['ventas_rolling7'] = df_agg['venta'].shift(1).rolling(window=7).mean()

### Eliminamos filas nulas

In [23]:
df_agg = df_agg.dropna().reset_index(drop=True)

In [24]:
df_agg.head()

,Date,LocationId,LocationStockId,Category,TipoTienda,DimensionesTienda,Ciudad,venta,Quantity,es_festivo,dia_semana,mes,sin_semana,cos_semana,sin_mes,cos_mes,ventas_lag1,ventas_lag7,ventas_rolling7
0,2023-01-02,1655,1655,FLORALES & HOMEOPATICOS,TIENDA,MEDIANO,SAMBORONDON,40.399320,1,1,0,1,0.0,1.0,0.5,0.866025,178.435201,70.334191,219.082863
1,2023-01-02,1655,1655,HOGAR - LIBROS & PAPELERIA,TIENDA,MEDIANO,SAMBORONDON,2.684240,1,1,0,1,0.0,1.0,0.5,0.866025,40.399320,458.181582,214.806452
2,2023-01-02,1655,1655,HUMANS - MEMORIALES,TIENDA,MEDIANO,SAMBORONDON,29.602040,1,1,0,1,0.0,1.0,0.5,0.866025,2.684240,27.512389,149.735404
3,2023-01-02,1655,1655,PASEAR,TIENDA,MEDIANO,SAMBORONDON,245.453963,12,1,0,1,0.0,1.0,0.5,0.866025,29.602040,54.921360,150.033925
4,2023-01-02,1655,1655,PRIMEROS AUXILIOS,TIENDA,MEDIANO,SAMBORONDON,13.451360,1,1,0,1,0.0,1.0,0.5,0.866025,245.453963,320.845865,177.252869


In [25]:
df_agg.dropna(inplace=True)
df_agg[df_agg.isna().all(axis=1)]

,Date,LocationId,LocationStockId,Category,TipoTienda,DimensionesTienda,Ciudad,venta,Quantity,es_festivo,dia_semana,mes,sin_semana,cos_semana,sin_mes,cos_mes,ventas_lag1,ventas_lag7,ventas_rolling7


### Exportamos el dataset listo para el modelo

In [26]:
df_agg.to_csv("../output/ModelDatasetMascotas.csv", index=False)